# IMDb(Internet Movie Database)のDataLoaderを実装

In [2]:
#パッケージのインポート
import glob
import os
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data
#import torchvision
#from torchvision import models,transforms
import cv2 
import time
import math
from matplotlib import cm

import gensim
import torchtext


## IMDbのデータをダウンロード

## IMDbデータセットをtsv形式に変換

In [3]:
#tsv形式のファイルにします
import glob
import os
import io
import string

In [4]:
#訓練データのtsvファイルを作成します
f=open("./data/IMDb_train.tsv","w")

path="./data/aclImdb/train/pos/"
for fname in glob.glob(os.path.join(path,"*txt")):
    with io.open(fname, "r", encoding="utf-8") as ff:
        text=ff.readline()

        #タブがあれば消しておきます
        text=text.replace("\t"," ")

        text=text+"\t"+"1"+"\t"+"\n"
        f.write(text)

path="./data/aclImdb/train/neg/"
for fname in glob.glob(os.path.join(path,"*txt")):
    with io.open(fname, "r", encoding="utf-8") as ff:
        text=ff.readline()

        #タブがあれば消しておきます
        text=text.replace("\t"," ")

        text=text+"\t"+"0"+"\t"+"\n"
        f.write(text)

f.close()

In [5]:
#テストデータの作成
f=open("./data/IMDb_test.tsv","w")

path="./data/aclImdb/test/pos/"
for fname in glob.glob(os.path.join(path,"*txt")):
    with io.open(fname, "r", encoding="utf-8") as ff:
        text=ff.readline()

        #タブがあれば消しておきます
        text=text.replace("\t"," ")

        text=text+"\t"+"1"+"\t"+"\n"
        f.write(text)

path="./data/aclImdb/test/neg/"
for fname in glob.glob(os.path.join(path,"*txt")):
    with io.open(fname, "r", encoding="utf-8") as ff:
        text=ff.readline()

        #タブがあれば消しておきます
        text=text.replace("\t"," ")

        text=text+"\t"+"0"+"\t"+"\n"
        f.write(text)

f.close()

## 前処理と単語分割の関数を定義

In [6]:
import string
import re

#以下の記号はスペースに置き換えます(カンマ、ピリオド除く)
#punctuationとは日本語で言う句点という意味です
print("区切り文字：",string.punctuation)
#区切り文字： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

区切り文字： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [7]:
type(string.punctuation)

str

In [8]:
#前処理
def preprocessing_text(text):
    #改行コードを除去
    text=re.sub("<br />"," ",text)

    #カンマ、ピリオド以外の記号をスペースに変換
    for p in string.punctuation:
        if (p==".")or(p==","):
            continue
        else:
            text=text.replace(p," ")

    #ピリオドなどの前後にはスペースを入れておく
    text=text.replace("."," .")
    text=text.replace(","," ")
    return text

In [9]:
#分かち書き(今回はデータが英語で、簡易的にスペースで区切る)
def tokenizer_punctuation(text):
    return text.strip().split()

In [10]:
#前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text=preprocessing_text(text)
    ret=tokenizer_punctuation(text)
    return ret

In [11]:
print(preprocessing_text("I like cats."))

I like cats .


In [12]:
#動作を確認します
print(tokenizer_with_preprocessing("I like cats."))

['I', 'like', 'cats', '.']


## DataLoaderの作成

In [13]:
#データを読み込んだ時に、読み込んだ内容に対して行う処理を定義します
import torchtext

#文章とラベルの両方に用意します
max_length=256

TEXT=torchtext.data.Field(sequential=True,tokenize=tokenizer_with_preprocessing,use_vocab=True,
lower=True,include_lengths=True,batch_first=True,fix_length=max_length,init_token="<cls>",eos_token="<eos>")

LABEL=torchtext.data.Field(sequential=False,use_vocab=False)

#引数の意味は次の通り
#init_token: 全部の文章で、文頭に入れておく単語
#eos_token: 全部の文章で、文末に入れておく単語

In [14]:
#フォルダ「data」から各tsvファイルを読み込ませます
train_val_ds,test_ds=torchtext.data.TabularDataset.splits(
    path="./data/",train="IMDb_train.tsv",test="IMDb_test.tsv",format="tsv",
    fields=[("Text",TEXT),("Label",LABEL)])

In [15]:
#動作確認
print("訓練及び検証のデータ数",len(train_val_ds))
print("1つ目の訓練及び検証のデータ",vars(train_val_ds[0]))

訓練及び検証のデータ数 25000
1つ目の訓練及び検証のデータ {'Text': ['for', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem', '.', 'imagine', 'a', 'movie', 'where', 'joe', 'piscopo', 'is', 'actually', 'funny', 'maureen', 'stapleton', 'is', 'a', 'scene', 'stealer', '.', 'the', 'moroni', 'character', 'is', 'an', 'absolute', 'scream', '.', 'watch', 'for', 'alan', 'the', 'skipper', 'hale', 'jr', '.', 'as', 'a', 'police', 'sgt', '.'], 'Label': '1'}


In [16]:
import random
#torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける

train_ds,val_ds=train_val_ds.split(split_ratio=0.8,random_state=random.seed(1234))

In [17]:
#動作確認
print("訓練データの数",len(train_ds))
print("検証データの数",len(val_ds))
print("1つ目の訓練データ",vars(train_ds[0]))

訓練データの数 20000
検証データの数 5000
1つ目の訓練データ {'Text': ['while', 'others', 'may', 'contend', 'that', 'by', 'viewing', 'other', 'works', 'by', 'bilal', 'one', 'will', 'better', 'appreciate', 'this', 'movie', 'it', 'does', 'fail', 'in', 'one', 'major', 'way', '.', 'it', 'does', 'not', 'stand', 'on', 'its', 'own', '.', 'the', 'plot', 'is', 'a', 'mishmash', 'that', 'is', 'confuses', 'symbolism', 'with', 'substance', '.', 'here', 's', 'an', 'idea', 'start', 'with', 'a', 'definite', 'story', '.', 'then', 'craft', 'symbolism', 'around', 'it', '.', 'we', 'start', 'with', 'two', 'different', 'narratives', 'this', 'female', 'that', 'is', 'somehow', 'turning', 'human', 'a', 'god', 'that', 'is', 'for', 'some', 'reason', 'being', 'judged', 'but', 'getting', 'one', 'last', 'fling', 'on', 'earth', 'and', 'this', 'mysterious', 'john', 'character', 'who', 'seems', 'to', 'be', 'developing', 'some', 'sort', 'of', 'resort', 'just', 'beyond', 'the', 'bounds', 'of', 'the', 'city', '.', 'why', 'none', 'of', 'these', 

## ボキャブラリーを作成

In [21]:
#torchtextで単語ベクトルとして英語学習済みモデルを読み込ませます
from torchtext.vocab import Vectors

english_fasttext_vectors=Vectors(name="./data/wiki-news-300d-1M.vec")

#単語ベクトルの中身を確認します
print("1単語を表現する次元数：",english_fasttext_vectors.dim)
print("単語数：",len(english_fasttext_vectors.itos))

1単語を表現する次元数： 300
単語数： 999994


In [22]:
#ベクトル化したバージョンのボキャブラリーを生成します
TEXT.build_vocab(train_ds,vectors=english_fasttext_vectors,min_freq=10)

#ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape) #18002個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

#ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi

torch.Size([18002, 300])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f946cd87390>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             'the': 4,
             '.': 5,
             'and': 6,
             'a': 7,
             'of': 8,
             'to': 9,
             'is': 10,
             'it': 11,
             'in': 12,
             'i': 13,
             'this': 14,
             'that': 15,
             's': 16,
             'was': 17,
             'as': 18,
             'for': 19,
             'movie': 20,
             'with': 21,
             'but': 22,
             'film': 23,
             't': 24,
             'on': 25,
             'you': 26,
             'not': 27,
             'he': 28,
             'are': 29,
             'his': 30,
             'have': 31,
             'be': 32,
             'one': 33,
             'all': 34,
             'at': 35,
             'they': 36,
             'by': 37

In [23]:
#DataLoaderを作成します(torchtextの文脈では単純にiteratorと呼ばれています)
train_dl=torchtext.data.Iterator(train_ds,batch_size=24,train=True)
val_dl=torchtext.data.Iterator(val_ds,batch_size=24,train=False,sort=False)
test_dl=torchtext.data.Iterator(test_ds,batch_size=24,train=False,sort=False)

In [24]:
#動作確認
#検証データのデータセットで確認
batch=next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[    2,    14,    10,  ...,     1,     1,     1],
        [    2,   137,   109,  ...,     1,     1,     1],
        [    2, 10125,  1057,  ...,     1,     1,     1],
        ...,
        [    2,    11,    16,  ...,     1,     1,     1],
        [    2,    13,    68,  ...,     6,     0,     3],
        [    2,    13,  1867,  ...,     1,     1,     1]]), tensor([176,  59, 243, 102, 137,  49, 192, 256, 145, 256, 253, 209, 256, 169,
        155, 233, 227, 256, 121, 151, 256, 148, 256, 197]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])
